In [1]:
# import the packages we need
import pandas as pd
import geopandas as gpd

# clean the data

We will take the crime data in 2016 as an example, the preparation for other year is similar.

In [ ]:
## read the original crime data in 2016
df = pd.read_csv('data_each_year//crime_16.csv')
print(f"Data frame is {df.shape[0]:,} x {df.shape[1]}")

In [ ]:
# choose the columns we need and rename them
df=df[['Primary Type','Year','Latitude','Longitude']]
df.rename(columns={'Primary Type':'type','Year':'year','Latitude':'lat','Longitude':'lon'},inplace=True)

In [ ]:
# check the data and drop NaN
df[df.type.isna()]

In [ ]:
df[df.lat.isna()]

In [ ]:
df.drop(df[df.lat.isna()].index.values, axis=0, inplace=True)

In [ ]:
# save the cleaned data
df.to_csv('data_each_year//2016.csv',index=False)

# prepare the data

prepare the boundary

In [5]:
# read the boundary
ward=gpd.read_file('crime_data//Boundaries Wards//Boundaries Wards//geo_export_6f1ca57a-6a70-4094-b008-70eace180952.shp')
ward=ward[['ward','geometry']]

In [9]:
# check crs
ward.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

# 1. The number of each crime type in each ward

We will take the crime data in 2016 as an example, the preparation for other year is similar. All we need to do is change the file name.

In [26]:
# read the cleaned data for 2016
df = pd.read_csv('data_each_year//2016.csv')
print(f"Data frame is {df.shape[0]:,} x {df.shape[1]}")

Data frame is 267,239 x 3


In [63]:
# convert dataframe to geodataframe
gdf = gpd.GeoDataFrame(df, 
      geometry=gpd.points_from_xy(df['lon'], df['lat'], crs='epsg:4326'))
gdf=gdf[['type','geometry']]

In [43]:
# check the crs
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [64]:
# add the point data to polygon
crime_in_wards =gpd.sjoin(gdf, ward, how="inner", op='intersects')
crime_in_wards

,type,geometry,index_right,ward
0,THEFT,POINT (-87.75466 41.95205),49,45
140,THEFT,POINT (-87.73318 41.95359),49,45
151,CRIMINAL TRESPASS,POINT (-87.77967 41.98655),49,45
209,THEFT,POINT (-87.73803 41.95250),49,45
211,ASSAULT,POINT (-87.74988 41.95819),49,45
...,...,...,...,...
208039,ROBBERY,POINT (-87.65154 41.95612),38,46
208054,DECEPTIVE PRACTICE,POINT (-87.65598 41.96307),38,46
208127,THEFT,POINT (-87.65354 41.95924),38,46
208186,THEFT,POINT (-87.66274 41.96774),38,46


In [65]:
len(crime_in_wards['type'].unique())

33

In [66]:
# calculate the number of each crime type in each ward
crime_type_ward = crime_in_wards.groupby(["ward", "type"]).agg({"index_right":"count"})
crime_type_ward.rename(columns={'index_right':'total_type_ward'},inplace=True)
crime_type_ward

total_type_ward
ward type                                              
1    ARSON                                            9
     ASSAULT                                        219
     BATTERY                                        452
     BURGLARY                                       188
     CONCEALED CARRY LICENSE VIOLATION                1
...                                                 ...
9    ROBBERY                                        220
     SEX OFFENSE                                     25
     STALKING                                         8
     THEFT                                          876
     WEAPONS VIOLATION                              406

[1275 rows x 1 columns]

In [67]:
#crime_type_ward.to_csv('preped-data//2016.csv')

In [68]:
# convert to pivot table
table=pd.pivot_table(crime_type_ward,index='ward',columns='type')

In [69]:
table

total_type_ward                           \
type           ARSON ASSAULT BATTERY BURGLARY   
ward                                            
1                9.0   219.0   452.0    188.0   
10              13.0   342.0   742.0    115.0   
11               8.0   242.0   431.0     98.0   
12              11.0   225.0   589.0     91.0   
13               1.0   134.0   296.0     84.0   
14              16.0   213.0   446.0    104.0   
15              22.0   325.0   792.0     77.0   
16              22.0   702.0  1542.0    273.0   
17              13.0   717.0  1689.0    280.0   
18               9.0   280.0   543.0    148.0   
19               1.0   131.0   226.0     81.0   
2                1.0   181.0   332.0    186.0   
20              22.0   692.0  1750.0    234.0   
21              21.0   640.0  1369.0    254.0   
22              11.0   251.0   586.0    103.0   
23              10.0   210.0   367.0     75.0   
24              29.0   746.0  2123.0    182.0   
25              14.0   326.0   614.0    132.0   
26              11.0   305.0   682.0    172.0   
27              16.0   639.0  1530.0    320.0   
28              35.0   837.0  2378.0    252.0   
29              27.0   548.0  1349.0    181.0   
3               25.0   581.0  1273.0    159.0   
30               8.0   189.0   498.0     85.0   
31               1.0   229.0   508.0     98.0   
32               5.0   113.0   204.0    164.0   
33               5.0   154.0   348.0     98.0   
34              19.0   618.0  1325.0    414.0   
35              10.0   206.0   454.0    102.0   
36               7.0   228.0   485.0     74.0   
37              23.0   595.0  1494.0    225.0   
38               5.0   127.0   281.0     86.0   
39               4.0   158.0   271.0    102.0   
4                5.0   516.0  1025.0    180.0   
40               1.0   174.0   449.0    110.0   
41               4.0   119.0   254.0     49.0   
42              15.0   370.0   968.0    498.0   
43               9.0    92.0   212.0    111.0   
44               5.0   146.0   291.0    130.0   
45               1.0   200.0   380.0    123.0   
46               3.0   219.0   458.0     89.0   
47               6.0   116.0   188.0    175.0   
48               4.0   187.0   462.0     91.0   
49               1.0   260.0   618.0    117.0   
5                8.0   513.0  1088.0    280.0   
50               4.0   169.0   423.0    131.0   
6               27.0   848.0  1937.0    321.0   
7               12.0   716.0  1591.0    294.0   
8               18.0   708.0  1615.0    406.0   
9               27.0   703.0  1399.0    319.0   

                                                                            \
type CONCEALED CARRY LICENSE VIOLATION CRIM SEXUAL ASSAULT CRIMINAL DAMAGE   
ward                                                                         
1                                  1.0                 1.0           340.0   
10                                 3.0                 1.0           490.0   
11                                 1.0                 NaN           357.0   
12                                 4.0                 NaN           328.0   
13                                 1.0                 NaN           281.0   
14                                 3.0                 NaN           309.0   
15                                 4.0                 2.0           484.0   
16                                 8.0                 2.0           850.0   
17                                 4.0                 2.0           894.0   
18                                 NaN                 NaN           369.0   
19                                 1.0                 2.0           193.0   
2                                  1.0                 NaN           192.0   
20                                 3.0                 3.0           865.0   
21                                 2.0                 3.0           936.0   
22                                 8.0                 3.0           303.0   
23 

In [70]:
# fill the NaN with 0, which means the number of this crime type in this ward is 0
table=table.fillna(0)
table

total_type_ward                           \
type           ARSON ASSAULT BATTERY BURGLARY   
ward                                            
1                9.0   219.0   452.0    188.0   
10              13.0   342.0   742.0    115.0   
11               8.0   242.0   431.0     98.0   
12              11.0   225.0   589.0     91.0   
13               1.0   134.0   296.0     84.0   
14              16.0   213.0   446.0    104.0   
15              22.0   325.0   792.0     77.0   
16              22.0   702.0  1542.0    273.0   
17              13.0   717.0  1689.0    280.0   
18               9.0   280.0   543.0    148.0   
19               1.0   131.0   226.0     81.0   
2                1.0   181.0   332.0    186.0   
20              22.0   692.0  1750.0    234.0   
21              21.0   640.0  1369.0    254.0   
22              11.0   251.0   586.0    103.0   
23              10.0   210.0   367.0     75.0   
24              29.0   746.0  2123.0    182.0   
25              14.0   326.0   614.0    132.0   
26              11.0   305.0   682.0    172.0   
27              16.0   639.0  1530.0    320.0   
28              35.0   837.0  2378.0    252.0   
29              27.0   548.0  1349.0    181.0   
3               25.0   581.0  1273.0    159.0   
30               8.0   189.0   498.0     85.0   
31               1.0   229.0   508.0     98.0   
32               5.0   113.0   204.0    164.0   
33               5.0   154.0   348.0     98.0   
34              19.0   618.0  1325.0    414.0   
35              10.0   206.0   454.0    102.0   
36               7.0   228.0   485.0     74.0   
37              23.0   595.0  1494.0    225.0   
38               5.0   127.0   281.0     86.0   
39               4.0   158.0   271.0    102.0   
4                5.0   516.0  1025.0    180.0   
40               1.0   174.0   449.0    110.0   
41               4.0   119.0   254.0     49.0   
42              15.0   370.0   968.0    498.0   
43               9.0    92.0   212.0    111.0   
44               5.0   146.0   291.0    130.0   
45               1.0   200.0   380.0    123.0   
46               3.0   219.0   458.0     89.0   
47               6.0   116.0   188.0    175.0   
48               4.0   187.0   462.0     91.0   
49               1.0   260.0   618.0    117.0   
5                8.0   513.0  1088.0    280.0   
50               4.0   169.0   423.0    131.0   
6               27.0   848.0  1937.0    321.0   
7               12.0   716.0  1591.0    294.0   
8               18.0   708.0  1615.0    406.0   
9               27.0   703.0  1399.0    319.0   

                                                                            \
type CONCEALED CARRY LICENSE VIOLATION CRIM SEXUAL ASSAULT CRIMINAL DAMAGE   
ward                                                                         
1                                  1.0                 1.0           340.0   
10                                 3.0                 1.0           490.0   
11                                 1.0                 0.0           357.0   
12                                 4.0                 0.0           328.0   
13                                 1.0                 0.0           281.0   
14                                 3.0                 0.0           309.0   
15                                 4.0                 2.0           484.0   
16                                 8.0                 2.0           850.0   
17                                 4.0                 2.0           894.0   
18                                 0.0                 0.0           369.0   
19                                 1.0                 2.0           193.0   
2                                  1.0                 0.0           192.0   
20                                 3.0                 3.0           865.0   
21                                 2.0                 3.0           936.0   
22                                 8.0                 3.0           303.0   
23 

In [71]:
table.to_csv('preped-data//table2016.csv')

# merge crime data from 2010 to 2020

read the crime data

In [ ]:
df2010=pd.read_csv('preped-data//table2010.csv')
df2010.columns = [str(col) + '_2010' for col in df2010.columns]
df2010=df2010.rename(columns={'ward_2010':'ward'})
df2010.sort_values(by='ward',inplace=True)

In [ ]:
df2011=pd.read_csv('preped-data//table2011.csv')
df2011.columns = [str(col) + '_2011' for col in df2011.columns]
df2011=df2011.rename(columns={'ward_2011':'ward'})
df2011.sort_values(by='ward',inplace=True)

In [ ]:
df2012=pd.read_csv('preped-data//table2012.csv')
df2012.columns = [str(col) + '_2012' for col in df2012.columns]
df2012=df2012.rename(columns={'ward_2012':'ward'})
df2012.sort_values(by='ward',inplace=True)

In [ ]:
df2013=pd.read_csv('preped-data//table2013.csv')
df2013.columns = [str(col) + '_2013' for col in df2013.columns]
df2013=df2013.rename(columns={'ward_2013':'ward'})
df2013.sort_values(by='ward',inplace=True)

In [ ]:
df2014=pd.read_csv('preped-data//table2014.csv')
df2014.columns = [str(col) + '_2014' for col in df2014.columns]
df2014=df2014.rename(columns={'ward_2014':'ward'})
df2014.sort_values(by='ward',inplace=True)

In [ ]:
df2015=pd.read_csv('preped-data//table2015.csv')
df2015.columns = [str(col) + '_2015' for col in df2015.columns]
df2015=df2015.rename(columns={'ward_2015':'ward'})
df2015.sort_values(by='ward',inplace=True)

In [ ]:
df2016=pd.read_csv('preped-data//table2016.csv')
df2016.columns = [str(col) + '_2016' for col in df2016.columns]
df2016=df2016.rename(columns={'ward_2016':'ward'})
df2016.sort_values(by='ward',inplace=True)

In [ ]:
df2017=pd.read_csv('preped-data//table2017.csv')
df2017.columns = [str(col) + '_2017' for col in df2017.columns]
df2017=df2017.rename(columns={'ward_2017':'ward'})
df2017.sort_values(by='ward',inplace=True)

In [ ]:
df2018=pd.read_csv('preped-data//table2018.csv')
df2018.columns = [str(col) + '_2018' for col in df2018.columns]
df2018=df2018.rename(columns={'ward_2018':'ward'})
df2018.sort_values(by='ward',inplace=True)

In [ ]:
df2019=pd.read_csv('preped-data//table2019.csv')
df2019.columns = [str(col) + '_2019' for col in df2019.columns]
df2019=df2019.rename(columns={'ward_2019':'ward'})
df2019.sort_values(by='ward',inplace=True)

In [ ]:
df2020=pd.read_csv('preped-data//table2020.csv')
df2020.columns = [str(col) + '_2020' for col in df2020.columns]
df2020=df2020.rename(columns={'ward_2020':'ward'})
df2020.sort_values(by='ward',inplace=True)

combine all the tables and save it as final table

In [ ]:
dfs = [df2010, df2011, df2012, df2013, df2014,df2015,df2016, df2017, df2018, df2019, df2020]
dfs = [df.set_index('ward') for df in dfs]
final_table=dfs[0].join(dfs[1:])
final_table.to_csv('preped-data//final_table.csv')

# calculate the top 10 crime types

take crime data in 2016 as an example

In [ ]:
df2016=pd.read_csv('preped-data//2016.csv')

In [ ]:
df2016=df2016[(df2016['type']=='THEFT')|(df2016['type']=='BATTERY')|(df2016['type']=='CRIMINAL DAMAGE') | (df2016['type']=='BURGLARY' )
| (df2016['type']=='DECEPTIVE PRACTICE' )|(df2016['type']=='ASSAULT')|(df2016['type']=='MOTOR VEHICLE THEFT')|(df2016['type']=='ROBBERY')
|(df2016['type']=='OTHER OFFENSE')|(df2016['type']=='NARCOTICS')]    

In [ ]:
# create a dataframe and record the top 10 crime type in each ward
top10=[]
for i in range(1,51):
    ward_top10=df2019[df2019['ward']==i].sort_values(by=['total_type_ward'],ascending=False).head(10)
    top10.append(ward_top10)
    i=i+1

In [ ]:
ward_top10= pd.concat(top10)
ward_top10

In [ ]:
# convert to pivot table
table=pd.pivot_table(ward_top10,index='ward',columns='type')
table=table.fillna(0)
table

In [ ]:
table.to_csv('preped-data//top10-table2016.csv')

In [ ]:
ward_top10.to_csv('preped-data//2016top.csv',index=False)

# merge the top 10 crime types

In [ ]:
# read the data
df2010=pd.read_csv('preped-data//top10-table2010.csv')
# add the year for each columns
df2010.columns = [str(col) + '_2010' for col in df2010.columns]
df2010=df2010.rename(columns={'ward_2010':'ward'})
df2010.sort_values(by='ward',inplace=True)

In [ ]:
df2011=pd.read_csv('preped-data//top10-table2011.csv')
df2011.columns = [str(col) + '_2011' for col in df2011.columns]
df2011=df2011.rename(columns={'ward_2011':'ward'})
df2011.sort_values(by='ward',inplace=True)

In [ ]:
df2012=pd.read_csv('preped-data//top10-table2012.csv')
df2012.columns = [str(col) + '_2012' for col in df2012.columns]
df2012=df2012.rename(columns={'ward_2012':'ward'})
df2012.sort_values(by='ward',inplace=True)

In [ ]:
df2013=pd.read_csv('preped-data//top10-table2013.csv')
df2013.columns = [str(col) + '_2013' for col in df2013.columns]
df2013=df2013.rename(columns={'ward_2013':'ward'})
df2013.sort_values(by='ward',inplace=True)

In [ ]:
df2014=pd.read_csv('preped-data//top10-table2014.csv')
df2014.columns = [str(col) + '_2014' for col in df2014.columns]
df2014=df2014.rename(columns={'ward_2014':'ward'})
df2014.sort_values(by='ward',inplace=True)

In [ ]:
df2015=pd.read_csv('preped-data//top10-table2015.csv')
df2015.columns = [str(col) + '_2015' for col in df2015.columns]
df2015=df2015.rename(columns={'ward_2015':'ward'})
df2015.sort_values(by='ward',inplace=True)

In [ ]:
df2016=pd.read_csv('preped-data//top10-table2016.csv')
df2016.columns = [str(col) + '_2016' for col in df2016.columns]
df2016=df2016.rename(columns={'ward_2016':'ward'})
df2016.sort_values(by='ward',inplace=True)

In [ ]:
df2017=pd.read_csv('preped-data//top10-table2017.csv')
df2017.columns = [str(col) + '_2017' for col in df2017.columns]
df2017=df2017.rename(columns={'ward_2017':'ward'})
df2017.sort_values(by='ward',inplace=True)

In [ ]:
df2018=pd.read_csv('preped-data//top10-table2018.csv')
df2018.columns = [str(col) + '_2018' for col in df2018.columns]
df2018=df2018.rename(columns={'ward_2018':'ward'})
df2018.sort_values(by='ward',inplace=True)

In [ ]:
df2019=pd.read_csv('preped-data//top10-table2019.csv')
df2019.columns = [str(col) + '_2019' for col in df2019.columns]
df2019=df2019.rename(columns={'ward_2019':'ward'})
df2019.sort_values(by='ward',inplace=True)

In [ ]:
df2020=pd.read_csv('preped-data//top10-table2020.csv')
df2020.columns = [str(col) + '_2020' for col in df2020.columns]
df2020=df2020.rename(columns={'ward_2020':'ward'})
df2020.sort_values(by='ward',inplace=True)

In [ ]:
dfs = [df2010, df2011, df2012, df2013, df2014,df2015,df2016, df2017, df2018, df2019, df2020]
dfs = [df.set_index('ward') for df in dfs]
final_table=dfs[0].join(dfs[1:])

In [ ]:
final_table.to_csv('preped-data//top10-final_table.csv')